In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import AdaBoostClassifier

# Завантаження даних
train_data = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_data.csv')
test_data = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_test.csv')

# Розділення на ознаки та цільову змінну
X = train_data.drop('y', axis=1)
y = train_data['y']

# Визначення числових та категоріальних ознак
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Створення конвеєра для числових змінних
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Створення конвеєра для категоріальних змінних
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Об'єднання конвеєрів для обробки всіх змінних
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення конвеєра з обробкою ознак, SMOTE та класифікатором AdaBoost
model = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', AdaBoostClassifier(random_state=42))
])

# Крос-валідація
cv_scores = cross_val_score(model, X, y, cv=5, scoring='balanced_accuracy')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean balanced accuracy: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

# Навчання моделі на всіх тренувальних даних
model.fit(X, y)

# Прогнози для тестового набору
test_predictions = model.predict(test_data)

# Створення файлу для відправки результатів
submission = pd.DataFrame({'index': test_data.index, 'y': test_predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("Файл submission.csv успішно створено.")